# Automatic import of Rome drinking fountains data to Wikidata
The following script downloads fountain data from osm as per https://github.com/water-fountains/proximap/issues/133, compares it to existing fountains in Wikidata for the same region, and creates Wikidata Quickstatement commands to complete the entries in Wikidata. New entities are created if no matching fountains are found.

## Initialize environment

In [48]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import io
import numpy as np
from urllib.request import urlopen
import json
from math import *
from platform import python_version
print("Python v "+python_version())
#https://github.com/paulhoule/gastrodon/issues/7 
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
from matplotlib import pyplot


191202_060659
Python v 3.6.5


In [49]:
#@prefix wikibase: <wikibase: <http://wikiba.se/ontology#> .
prefixes=inline("""
   @prefix wd: <http://www.wikidata.org/entity/> .
   @prefix wdt: <http://www.wikidata.org/prop/direct/> .
   @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
   @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
""").graph
endpoint=RemoteEndpoint(
   #"https://query.wikidata.org/sparql"
    "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
   ,prefixes=prefixes
)

## Load data

In [50]:
df = pd.read_csv("osmFountainsRome191201_082705.csv")

In [51]:
df.head()

,type,id,lat,lon,amenity,fountain,created_by,wheelchair,flow,architect,...,operator,designation,level,location,covered,description:de,website,artwork_type,subject:wikidata,drinking_water:legal
0,node,246569213,41.824508,12.485546,drinking_water,nasone,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,246569214,41.852621,12.478328,drinking_water,nasone,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,246569215,41.854102,12.476834,drinking_water,nasone,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,246569216,41.863749,12.478948,drinking_water,nasone,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,246571139,41.904287,12.513278,drinking_water,NaN,JOSM,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Rename columns to make them easier to work with

In [52]:
# remove not needed columns
df = df.drop(columns=['type','id'])

In [53]:
# rename columns
df = df.rename(index=str, columns=
               {"lat": "Y"})

In [68]:
# rename columns
df = df.rename(index=str, columns=
               {"lon": "X"})

In [69]:
df.head()

,Y,X,amenity,fountain,created_by,wheelchair,flow,architect,description,drinking_water,...,drinking_water:legal,nearest_qid,nearest_distance,2nd_nearest_distance,nearest_has_label_de,nearest_has_date,nearest_has_operator,nearest_has_code,nearest_has_water_type,match_found
0,41.824508,12.485546,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76937644,0.0,424.336166,True,False,False,False,False,match
1,41.852621,12.478328,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76937896,0.0,129.883666,True,False,False,False,False,match
2,41.854102,12.476834,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76937912,0.0,102.431975,True,False,False,False,False,match
3,41.863749,12.478948,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76937931,0.0,254.841443,True,False,False,False,False,match
4,41.904287,12.513278,drinking_water,NaN,JOSM,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76937949,0.0,116.905101,True,False,False,False,False,match


In [70]:
len(df)

1645

## Identify already existing fountains
### Query fountains from Wikidata

In [71]:
# Find the geographic extent of the data

buffer = 0.0003  # in degrees, corresponds to about 20-30 meters)
bounds = {
    'minX': df['X'].min() - buffer,
    'minY': df['Y'].min() - buffer,
    'maxX': df['X'].max() + buffer,
    'maxY': df['Y'].max() + buffer
}
print("bounds: ")
for key,value in bounds.items():
    print(key+ ": "+str(value))

bounds: 
minX: 12.3238906
minY: 41.802459999999996
maxX: 12.6220409
maxY: 41.99526720000001


In [72]:
# Query fountains (both water wells and fountains) from Wikidata within bounding box found above
# placeLabel 

query_string = """SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{{
  # Enter coordinates
  SERVICE wikibase:box {{
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point({minX} {minY})"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point({maxX} {maxY})"^^geo:wktLiteral.
  }} .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q43483 }} || EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q483453 }}).
  SERVICE wikibase:label {{
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  }} 
  OPTIONAL {{ ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}}
  OPTIONAL {{ ?place wdt:P571 ?date.}}
  OPTIONAL {{ ?place wdt:P5623 ?water_supply_type}}
  OPTIONAL {{ ?place wdt:P137 ?operator.}}
  OPTIONAL {{ ?place rdfs:placeLabel ?placeLabel}}
}}
  """.format(**bounds)

print(query_string)

SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{
  # Enter coordinates
  SERVICE wikibase:box {
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point(12.3238906 41.802459999999996)"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point(12.6220409 41.99526720000001)"^^geo:wktLiteral.
  } .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS { ?place wdt:P31/wdt:P279* wd:Q43483 } || EXISTS { ?place wdt:P31/wdt:P279* wd:Q483453 }).
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  } 
  OPTIONAL { ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}
  OPTIONAL { ?place wdt:P571 ?date.}
  OPTIONAL { ?place wdt:P5623 ?water_supply_type}
  OPTIONAL { ?place wdt:P137 ?operator.}
  OPTIONAL { ?place rdfs:placeLabel ?placeLabel}
}
  


In [73]:
# Perform query
query_result = endpoint.select(query_string)

In [74]:
#print(query_string)
print("\n\nTotal number of rows incl. duplicates "+str(len(query_result))+" size "+str(query_result.size))



Total number of rows incl. duplicates 1682 size 13456


### Tidy up data

In [75]:
# Extract coordinates from Wikidata results

query_result['X'] = query_result['location'].apply(lambda l:float(l.split('(')[1].split(' ')[0]))
query_result['Y'] = query_result['location'].apply(lambda l:float(l.split(' ')[1].split(')')[0]))

In [76]:
query_result.head(100)

,place,placeLabel,location,date,catalog_code,catalogLabel,operator,water_supply_type,X,Y
0,wd:Q74574057,Brunnen,Point(12.519195 41.8814447),None,None,None,None,None,12.519195,41.881445
1,wd:Q76937644,Brunnen,Point(12.485546 41.8245081),None,None,None,None,None,12.485546,41.824508
2,wd:Q76937896,Brunnen,Point(12.4783285 41.8526205),None,None,None,None,None,12.478328,41.852621
3,wd:Q76937912,Brunnen,Point(12.4768335 41.8541022),None,None,None,None,None,12.476833,41.854102
4,wd:Q76937931,Brunnen,Point(12.4789476 41.8637486),None,None,None,None,None,12.478948,41.863749
...,...,...,...,...,...,...,...,...,...,...
95,wd:Q76939564,Brunnen,Point(12.4849325 41.9013552),None,None,None,None,None,12.484932,41.901355
96,wd:Q76939583,Brunnen,Point(12.485754 41.9016622),None,None,None,None,None,12.485754,41.901662
97,wd:Q76939597,Brunnen,Point(12.487827 41.9114186),None,None,None,None,None,12.487827,41.911419
98,wd:Q76939614,Brunnen,Point(12.5126101 41.8923832),None,None,None,None,None,12.512610,41.892383


In [61]:
# show duplicates
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html#pandas.DataFrame.duplicated

ids = query_result['placeLabel']
#ids = query_result['label']
duplicates = query_result[ids.isin(ids[ids.duplicated(keep=False)])]
dupli = duplicates.drop(columns=['catalog_code', 'catalogLabel', 'water_supply_type', 'date', 'catalogLabel', 'location', 'operator'])
dupli.sort_values(by=['placeLabel'],inplace=True)
print ("Duplicates: "+str(len(dupli))+"\n\n")
print(dupli.to_string())

Duplicates: 1602


             place                               placeLabel          X          Y
0     wd:Q74574057                                  Brunnen  12.519195  41.881445
1079  wd:Q76975647                                  Brunnen  12.456227  41.900388
1078  wd:Q76968477                                  Brunnen  12.600466  41.805598
1077  wd:Q76968460                                  Brunnen  12.560869  41.854824
1076  wd:Q76968445                                  Brunnen  12.580297  41.914074
1075  wd:Q76968426                                  Brunnen  12.490243  41.913317
1074  wd:Q76968409                                  Brunnen  12.399064  41.957969
1073  wd:Q76968389                                  Brunnen  12.472559  41.854820
1072  wd:Q76968373                                  Brunnen  12.513590  41.832478
1071  wd:Q76968353                                  Brunnen  12.463766  41.893189
1080  wd:Q76975667                                  Brunnen  12.482713  41.9148

In [62]:
def createMerge(dupli, extraText):
    #Create MERGE Command
    linesDup = []
    prevQ= ""
    prevX=0
    prevY=0
    for index, row in dupli.iterrows():
        
        # either create new or edit existing entity
        if row['X'] == prevX:
            if row['Y'] == prevY:
                lineDup = "MERGE\t"+prevQ[3:]+"\t"+row['place'][3:]+"\n"
                linesDup.append(lineDup)
            else:
                prevX=row['X']
                prevY=row['Y']
                prevQ=row['place']
        else:
            prevX=row['X']
            prevY=row['Y']
            prevQ=row['place']
            
    print("Merge commands"+extraText+" total: "+str(len(linesDup))+"\n\n")
    print(linesDup)
    
createMerge(dupli, "")    

Merge commands total: 0


[]


In [130]:
#write MERGE command to File
quickStatDupliFileName = "quickstatement_commands_Hamburg_drink_DUPLI_"+dt.now().strftime(dtFmt)+".txt"
with io.open(quickStatDupliFileName, "w", encoding='utf8') as f:
    f.writelines(linesDup)
print("wrote '"+quickStatDupliFileName+"' with "+str(len(linesDup))+" lines")

NameError: name 'linesDup' is not defined

In [37]:
# show duplicates STRICT
#idsS = query_result['placeLabel','X','Y'] #this does not work yet
#duplicatesS = query_result[ids.isin(ids[ids.duplicated(keep=False)])]
#dupliS = duplicatesS.drop(columns=['catalog_code', 'catalogLabel', 'water_supply_type', 'date', 'catalogLabel', 'location', 'operator'])
#dupliS.sort_values(by=['placeLabel'],inplace=True)
#print(dupliS.to_string())
#createMerge(dupliS, " - strict")  

In [132]:
# remove duplicate entries

# duplicate entries are caused when e.g. a fountain has catalog codes from two catalogs
query_result = query_result.drop_duplicates('place')
print("\n\nTotal number of rows without duplicates "+str(len(query_result)))



Total number of rows without duplicates 130


In [133]:
query_result.head(100)

,place,placeLabel,location,date,catalog_code,catalogLabel,operator,water_supply_type,X,Y
0,wd:Q1454814,Minervabrunnen,Point(9.95166667 53.54583333),None,None,None,None,None,9.951667,53.545833
1,wd:Q814528,Behnbrunnen,Point(9.94194444 53.54805556),None,None,None,None,None,9.941944,53.548056
2,wd:Q2358953,Stuhlmannbrunnen,Point(9.93527778 53.54972222),None,None,None,None,None,9.935278,53.549722
3,wd:Q2651496,Alsterfontäne,Point(9.99495278 53.55461667),None,None,None,None,None,9.994953,53.554617
4,wd:Q18632417,Vierländerin-Brunnen,Point(9.99014 53.54766),None,None,None,None,None,9.990140,53.547660
...,...,...,...,...,...,...,...,...,...,...
95,wd:Q76308142,Q76308142,Point(10.2961728 53.4268728),None,None,None,None,None,10.296173,53.426873
96,wd:Q76308157,Q76308157,Point(9.9418923 53.548183),None,None,None,None,None,9.941892,53.548183
97,wd:Q76308173,Q76308173,Point(10.0831994 53.626006),None,None,None,None,None,10.083199,53.626006
98,wd:Q76308188,Q76308188,Point(9.9919829 53.5509746),None,None,None,None,None,9.991983,53.550975


### Compute distances between fountains

In [77]:
# helper function to compute distances on the globe, returns distances in meters
def spherical_dist(pos1, pos2, r=6371000):
    pos1 = pos1 * np.pi / 180
    pos2 = pos2 * np.pi / 180
    cos_lat1 = np.cos(pos1[..., 0])
    cos_lat2 = np.cos(pos2[..., 0])
    cos_lat_d = np.cos(pos1[..., 0] - pos2[..., 0])
    cos_lon_d = np.cos(pos1[..., 1] - pos2[..., 1])
    return r * np.arccos(cos_lat_d - cos_lat1 * cos_lat2 * (1 - cos_lon_d))


# compute distances from each ODZ fountain to each Wikidata fountain
distances = spherical_dist(df[['X','Y']].values[:, None], query_result[['X','Y']].values)

### Identify nearest and second nearest matches for each osm fountain

In [78]:
# indexes of nearest fountains
nearest_idx = np.argmin(distances, axis=1).tolist()

# QID of nearest fountains
df['nearest_qid'] = query_result.iloc[nearest_idx]['place'].apply(lambda id:id[3:]).tolist()

# distance to nearest fountain
df['nearest_distance'] = np.min(distances, axis=1).tolist()


# then remove nearest
i_line=0
for i_col in nearest_idx:
    distances[i_line, i_col] = 100000
    i_line += 1
# find distance to second nearest
df['2nd_nearest_distance'] = np.min(distances, axis=1).tolist()

df.head(100)

,Y,X,amenity,fountain,created_by,wheelchair,flow,architect,description,drinking_water,...,drinking_water:legal,nearest_qid,nearest_distance,2nd_nearest_distance,nearest_has_label_de,nearest_has_date,nearest_has_operator,nearest_has_code,nearest_has_water_type,match_found
0,41.824508,12.485546,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76937644,0.0,424.336166,True,False,False,False,False,match
1,41.852621,12.478328,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76937896,0.0,129.883666,True,False,False,False,False,match
2,41.854102,12.476834,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76937912,0.0,102.431975,True,False,False,False,False,match
3,41.863749,12.478948,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76937931,0.0,254.841443,True,False,False,False,False,match
4,41.904287,12.513278,drinking_water,NaN,JOSM,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76937949,0.0,116.905101,True,False,False,False,False,match
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,41.901662,12.485754,drinking_water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76939583,0.0,93.295995,True,False,False,False,False,match
96,41.911419,12.487827,drinking_water,NaN,JOSM,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76939597,0.0,338.638027,True,False,False,False,False,match
97,41.892383,12.512610,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76939614,0.0,79.473433,True,False,False,False,False,match
98,41.893211,12.509447,drinking_water,nasone,NaN,NaN,push-button,NaN,NaN,NaN,...,NaN,Q76939633,0.0,139.503504,True,False,False,False,False,match


### Find out what information already exists for the nearest fountains

In [79]:
# does nearest have label in german?
df['nearest_has_label_de'] = (query_result.iloc[nearest_idx]['place'].apply(lambda p:p[3:]) != query_result.iloc[nearest_idx]['placeLabel']).tolist()

# does nearest have date?
df['nearest_has_date'] = query_result.iloc[nearest_idx]['date'].apply(lambda d:d is not None).tolist()

# does nearest have operator?
df['nearest_has_operator'] = query_result.iloc[nearest_idx]['operator'].apply(lambda id:id is not None).tolist()

# does nearest have catalog code?
df['nearest_has_code'] = query_result.iloc[nearest_idx]['catalog_code'].apply(lambda id:id is not None).tolist()

# does nearest have water type?
df['nearest_has_water_type'] = query_result.iloc[nearest_idx]['water_supply_type'].apply(lambda id:id is not None).tolist()

### Decide on whether nearest fountain should be considered a match

In [80]:
# The nearest fountain is a match if: 
# - no further than x m away
# - 2nd nearest fountain at nearest least ratio_min further away than the nearest fountain
def validate_proposal(qid, d1, d2, dmax=10, ratio_min=0.5):
    
    if d1 == 0 or (d1<=dmax and d2/d1-1 >= ratio_min):
        return 'match'
    elif d1<=dmax and d2/d1-1 < ratio_min:
        return 'unclear'
    else:
        print("Q-# "+qid+"\td1 "+str(d1)+"\td2 "+str(d2))
        return 'no match'
    
for index, row in df.iterrows():
    df.loc[index, 'match_found'] = validate_proposal(
        row['nearest_qid'], 
        row['nearest_distance'], 
        row['2nd_nearest_distance'],
        dmax=15
    )
dffinal = df.drop(columns=['nearest_distance', '2nd_nearest_distance'])

In [81]:
len(dffinal)

1645

In [82]:
dffinal

,Y,X,amenity,fountain,created_by,wheelchair,flow,architect,description,drinking_water,...,artwork_type,subject:wikidata,drinking_water:legal,nearest_qid,nearest_has_label_de,nearest_has_date,nearest_has_operator,nearest_has_code,nearest_has_water_type,match_found
0,41.824508,12.485546,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76937644,True,False,False,False,False,match
1,41.852621,12.478328,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76937896,True,False,False,False,False,match
2,41.854102,12.476834,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76937912,True,False,False,False,False,match
3,41.863749,12.478948,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76937931,True,False,False,False,False,match
4,41.904287,12.513278,drinking_water,NaN,JOSM,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76937949,True,False,False,False,False,match
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1640,41.913437,12.461680,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76978171,True,False,False,False,False,match
1641,41.902114,12.590335,drinking_water,NaN,NaN,yes,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76978191,True,False,True,False,False,match
1642,41.888007,12.461600,drinking_water,nasone,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76978256,True,False,False,False,False,match
1643,41.903375,12.512719,drinking_water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76978310,True,False,False,False,False,match


In [21]:
print("\n\nTotal number of rows "+str(len(dffinal)))



Total number of rows 1645


In [84]:
#write osm to csv
osmDumpFileName = "osm_dump_Rome_fountains_"+dt.now().strftime(dtFmt)+".csv"
dffinal.to_csv(osmDumpFileName)
print("wrote '"+osmDumpFileName+"' ")

wrote 'osm_dump_Rome_fountains_191202_061854.csv' 


## Create Quickstatement commands from data
### Helper functions to format content according to Quickstatements v1 syntax

In [45]:
def process_coordinates(x, y):
    # format geographic coordinates
    return '@{1:1.8f}/{0:1.8f}'.format(x,y)


def process_year(date):
    # format date
    if np.isnan(date):
        return ''
    else:
        return '+{0:4d}-00-00T00:00:00Z/9'.format(int(date))

    
fountain_type_map = {
    'öffentlicher Brunnen': 'Q53628296',
    'Notwasserbrunnen': 'Q53628522',
    'privater Brunnen': 'Q53629707',
    'Brunnen in städtischer Liegenschaft': 'Q53628618',
    'Brunnen des Verschönerungsvereins': 'Q53628761',
    'Brunnen mit eigener Versorgung': 'Q53630002'
}

water_type_map = {
    'Verteilnetz': 'Q53633635',
    'Quellwasser': 'Q1881858',
    'eigene Versorgung': 'Q53634173',
    'Grundwasser': 'Q161598'
}

def process_fountain_type(type):
    # translate fountain types to wikidata values
    return fountain_type_map[type]


def process_water_type(type):
    # translate water types to wikidata values
    return water_type_map[type]


def process_label(text, dfltLabel):
    # process German language labels
    #print (text + ' '+dfltLabel)
    if text is None:
        return dfltLabel
    elif 'nan' == text.lower():
        return '"'+dfltLabel+'"'
    elif 'none' == text.lower():
        return '"'+dfltLabel+'"'.format(text)
    elif 'brunnen' in text.lower():
        return '"{}"'.format(text)
    elif 'fountain' in text.lower():
        return '"{}"'.format(text)
    else:
        return '"'+dfltLabel+' ({})"'.format(text)

    
def process_label_de(text):
    dfltLabel = 'Brunnen'
    return process_label(text, dfltLabel)
    
def process_label_en(text):
    dfltLabel = 'Fountain'
    return process_label(text, dfltLabel)
    

def createline(lines, item, prop, value, extra, qualifiers=[]):
    # general function to create Quickstatement v1 commands
    if value != '' and value != '""':
        statement = '{}\t{}\t{}'.format(item, prop, value)
        if len(qualifiers):
            # append qualifiers if applicable
            for q in qualifiers:
                statement += '\t{}\t{}'.format(q['prop'], q['value'])
        statement += extra
        statement += '\n'
        lines.append(statement)
    return lines

### Create statements, taking care not to overwrite existing data

In [46]:
# initialize command storage list
lines = []
statedId = "\tS248\tQ1224853"
i=0
for index, row in dffinal.iterrows():
    i+=1
    # either create new or edit existing entity
    if row['match_found'] == 'no match':
        # create a new fountain
        lines.append('CREATE\n')
        item = 'LAST'
    elif row['match_found'] == 'unclear':
        print('unclear match')
        print(row)
        continue
    elif row['match_found'] == 'match':
        # update existing fountain
        item = row['nearest_qid']
        
        
    # Add this basic information only if creating a new entity
    if item == 'LAST':
        # instance of  fountain
        lines = createline(lines, item, 'P31', 'Q483453',statedId)

        # coordinates
        lines = createline(lines, item, 'P625', process_coordinates(row['X'], row['Y']),statedId)
        
    # For other properties, add information if the entity is new or if property does not yet exist
    
    # label in german
    if item == 'LAST' or not row['nearest_has_label_de']:
        lines = createline(lines, item, 'Lde', process_label_de(str(row['name'])),statedId)

    # label in english
    #if item == 'LAST' or not row['nearest_has_label_en']:
    #    lines = createline(lines, item, 'Len', process_label_en(str(row['name'])),statedId)
        
    # creation date
    #if item == 'LAST' or not row['nearest_has_date']:
    #    lines = createline(lines, item, 'P571', process_year(row['date']))

    # operated by  Hamburg Wasser
    #print(str(i)+": "+str(row['operator']))
    if  pd.isna(row['operator']):
        print(str(i)+": "+"operator none")
    else:
         lines = createline(lines, item, 'P137', 'Q1572943',statedId)
            
    # catalog number can always be added (it is hard to check for)
    #lines = createline(lines, item, 'P528', '"{}"'.format(row['operator_id']), [{
    #    'prop': 'P972',
    #    'value': 'Q53629101'
    #}])

1: operator none
2: operator none
3: operator none
4: operator none
5: operator none
6: operator none
7: operator none
8: operator none
9: operator none
10: operator none
11: operator none
12: operator none
13: operator none
14: operator none
15: operator none
16: operator none
17: operator none
18: operator none
19: operator none
20: operator none
21: operator none
22: operator none
23: operator none
24: operator none
25: operator none
26: operator none
27: operator none
28: operator none
29: operator none
30: operator none
31: operator none
32: operator none
33: operator none
34: operator none
35: operator none
36: operator none
37: operator none
38: operator none
39: operator none
40: operator none
41: operator none
42: operator none
43: operator none
44: operator none
45: operator none
46: operator none
47: operator none
48: operator none
49: operator none
50: operator none
51: operator none
52: operator none
53: operator none
54: operator none
55: operator none
56: operator none
5

608: operator none
609: operator none
610: operator none
611: operator none
612: operator none
613: operator none
614: operator none
615: operator none
616: operator none
617: operator none
618: operator none
619: operator none
620: operator none
621: operator none
622: operator none
623: operator none
624: operator none
625: operator none
626: operator none
627: operator none
628: operator none
629: operator none
630: operator none
631: operator none
632: operator none
633: operator none
634: operator none
635: operator none
636: operator none
637: operator none
638: operator none
639: operator none
640: operator none
641: operator none
642: operator none
643: operator none
644: operator none
645: operator none
646: operator none
647: operator none
648: operator none
649: operator none
650: operator none
651: operator none
652: operator none
653: operator none
654: operator none
655: operator none
656: operator none
657: operator none
658: operator none
659: operator none
660: operato

1250: operator none
1251: operator none
1252: operator none
1253: operator none
1254: operator none
1255: operator none
1256: operator none
1257: operator none
1258: operator none
1259: operator none
1260: operator none
1261: operator none
1262: operator none
1263: operator none
1264: operator none
1265: operator none
1266: operator none
1267: operator none
1268: operator none
1269: operator none
1270: operator none
1271: operator none
1272: operator none
1273: operator none
1274: operator none
1275: operator none
1276: operator none
1277: operator none
1278: operator none
1279: operator none
1280: operator none
1281: operator none
1282: operator none
1283: operator none
1284: operator none
1285: operator none
1286: operator none
1287: operator none
1288: operator none
1289: operator none
1290: operator none
1291: operator none
1292: operator none
1293: operator none
1294: operator none
1295: operator none
1296: operator none
1297: operator none
1298: operator none
1299: operator none


# Write commands to file

In [47]:
quickStatFileName = "quickstatement_commands_Rome_fountain_"+dt.now().strftime(dtFmt)+".txt"
with io.open(quickStatFileName, "w", encoding='utf8') as f:
    f.writelines(lines)
print("wrote '"+quickStatFileName+"' with "+str(len(lines))+" lines")

wrote 'quickstatement_commands_Rome_fountain_191201_085501.txt' with 8191 lines


# Import into Wikidata
- Go to https://tools.wmflabs.org/wikidata-todo/quick_statements.php.
- Authenticate yourself with your Wikidata account.
- Copy and paste the contents of quickstatement_commands*.txt into the blank field, and run the commands

see ../20191030_1600_import.png

...
58. Processing Q72935495 (Q72935495 Lde "Brunnen (Seelöwe-Planschbecken )")
59. Processing Q72935495 (Q72935495 P137 Q27229237)

All done!.

In [15]:
# it may well take half an hour until it works https://query.wikidata.org/